In [ ]:
import numpy as np
import pickle
import sys
import os
import gzip
from tqdm import tqdm

sys.path.append(os.path.abspath(os.path.join("..")))

import engine
import utils

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
path = "stimuli/ground_truth/"
world_files = os.listdir(path)
world_numbers = sorted([int(wf[6:-5]) for wf in world_files])

In [4]:
drop_noise = 0.3
col_mean = 0.0
col_sd = 0.7

In [5]:
worlds = []

for world_num in world_numbers:
    world = utils.load_trial(world_num, drop_noise=drop_noise, col_mean=col_mean, col_sd=col_sd)
    worlds.append((world_num, world))

In [ ]:
np.random.seed(1)
nsims = 1000
saved_sims = {}

for world_num, world in tqdm(worlds):
    # print("World", world_num)
    world_sims = []
    for hole in [0,1,2]:
        world["hole_dropped_into"] = hole
        hole_sims = []
        for i in range(nsims):
            sim = engine.run_simulation(world, convert_coordinates=True)
            hole_sims.append(sim)
        world_sims.append(hole_sims)
    saved_sims[world_num] = world_sims

In [ ]:
saved_sims_sum = {}
for world_num, sims in saved_sims.items():
    world_sum = []
    for hole in [0,1,2]:
        hole_sims = sims[hole]
        hole_sum = []
        for sim in hole_sims:
            sim_sum = {}
            sim_sum["drop"] = sim["drop"]
            sim_sum["collisions"] = sim["collisions"]
            sim_sum["ball_position"] = [sim["ball_position"][-1]]
            hole_sum.append(sim_sum)
        world_sum.append(hole_sum)
    saved_sims_sum[world_num] = world_sum

In [ ]:
with gzip.open("saved_sims/saved_sims_sum_nsims_1000_drop_noise_0.3_col_mean_0.0_col_sd_0.7.gz", "wb") as gz_file:
    pickle.dump(saved_sims_sum, gz_file)